In [1]:
#the purpose of this piece of code is to evaluate the model using 
#testing set, and then visualize the results by drawing the output
#bounding boxes on the image and then output the results in .csv format

#loading the model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image    

In [2]:
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has 3 classes - text/non-text and background
num_classes = 3

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)

PATH= "./current_model_nms_05_firm"
model.load_state_dict(torch.load(PATH))

# move model to the right device
model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
    

In [3]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image

#setting up the same dataset format as the training one
class BCEDetectDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path).convert("RGB")

        mask = np.array(mask)
        
        mask_map = mask[:,:,0]
        labels = []
        obj_ids = np.unique(mask[:,:,0])
        obj_ids = obj_ids[1:]
        for each_unique in obj_ids:
            x,y = np.where(mask[:,:,0]==each_unique)
            labels.append(mask[x[0],y[0],1])
        labels = torch.tensor(labels,dtype = torch.int64)

        masks = mask_map == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
#         labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)
    
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

dataset_test = BCEDetectDataset('bce_testing/', get_transform(train=False))

In [4]:
#sending the testing images to the model for outputs
import numpy as np
import matplotlib.pyplot as plt

img, _ = dataset_test[0]

image_list = []
boxes_list = []
scores_list = []
labels_list = []

model.eval()
for i in range(len(dataset_test)):
#     print(i)
    img,_ = dataset_test[i]
    image_list.append(img.mul(255).permute(1, 2, 0).byte().numpy())
    with torch.no_grad():
        prediction = model([img.to(device)])# pick one image from the test set
        boxes_list.append(prediction[0]['boxes'])
        scores_list.append(prediction[0]['scores'])
        labels_list.append(prediction[0]['labels'])
        
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
import cv2
    
drawing_image_list = []
boxes_accepted_list = []

#drawing output bounding boxes on the images for visualization of results
for i in range(len(boxes_list)):
    sample_image = image_list[i]
    drawing_image = np.copy(sample_image)
    boxes = boxes_list[i]
    scores = scores_list[i]
    labels = labels_list[i]
    boxes_accepted = []
#     print(labels)
    for j in range(len(boxes)):
        if(scores[j]>0.8): #an arbitrary decided value, given during 
            #validation step
            xmin = boxes[j][0]
            ymin = boxes[j][1]
            xmax = boxes[j][2]
            ymax = boxes[j][3]
            start_point = (xmin,ymin)
            end_point = (xmax,ymax)
            color_text = (255, 0, 0)
            color_picture = (0, 255, 0)
            thickness = 3
            if(labels[j]==1):
                drawing_image = cv2.rectangle(drawing_image, start_point, end_point, color_text, thickness) 
            elif(labels[j]==2):
                drawing_image = cv2.rectangle(drawing_image, start_point, end_point, color_picture, thickness) 
#             print(boxes[j])
            
            boxes_accepted.append(boxes[j])
    boxes_accepted_list.append(boxes_accepted)
    drawing_image_list.append(drawing_image)


In [6]:
import csv
#write the results into .csv format so that they can be processed later on
def write_box_info_into_csv(boxes_accepted_list,labels_list,saving_root_boxes):
    number_in_sorted_order = list(range(len(boxes_accepted_list)))
    file_ordered_list = []
    for n in number_in_sorted_order:
        file_ordered_list.append(str(n))
    file_ordered_list = sorted(file_ordered_list) 
    
    for i in range(len(boxes_accepted_list)):
        with open(saving_root_boxes+file_ordered_list[i]+'_boxes.csv', 'w', newline='') as csvfile:
            fieldnames = ['xmin', 'ymin','xmax','ymax','label']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for j in range(len(boxes_accepted_list[i])):
                xmin = boxes_accepted_list[i][j][0]
                ymin = boxes_accepted_list[i][j][1]
                xmax = boxes_accepted_list[i][j][2]
                ymax = boxes_accepted_list[i][j][3]
                writer.writerow({'xmin': str(int(xmin)), 'ymin': str(int(ymin)),'xmax': str(int(xmax)),'ymax': str(int(ymax)),'label':str(int(labels_list[i][j]))})

In [7]:
#record images and boxes
#the sorting is to make sure the file order is from xxx_0.png to xxx_n.png
saving_root_images = "BCE_test/result_images/"
saving_root_boxes = "BCE_test/result_boxes/"
number_of_images = 300
number_in_sorted_order = sorted(list(range(0,number_of_images)))
file_ordered_list = []
for n in number_in_sorted_order:
    file_ordered_list.append(str(n))
file_ordered_list = sorted(file_ordered_list)

for i in range(len(drawing_image_list)):
    cv2.imwrite(saving_root_images+file_ordered_list[i]+".png",drawing_image_list[i])
    
write_box_info_into_csv(boxes_accepted_list,labels_list,saving_root_boxes = "BCE_test/result_boxes/")